In [2]:
# import packages and define functions
import cartopy 
import cartopy.crs as ccrs
from global_land_mask import globe
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyproj
from scipy.ndimage.measurements import label
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon
import warnings
#warnings.filterwarnings('ignore')
import xarray as xr

# initialize custom function
    # calculate the approximate distance between two latitude, longitude coordinates
def estimate_dist_ll2(lat1, lon1, lat2, lon2):
    R_earth = 6371 # radius of earth km
    lat1, lon1, lat2, lon2 = np.deg2rad([lat1, lon1, lat2, lon2])
    d = np.sin((lat2 - lat1)/2)**2 + np.cos(lat1)*np.cos(lat2) * np.sin((lon2 - lon1)/2)**2
    return 2 * R_earth * np.arcsin(np.sqrt(d))

    # detect ARs, input east and north IVT vectors as function of lat, lon, and time,
    # and the corresponding lat, lon, and time (datetime64) arrays
    # follows the methodology of Guan, B., & Waliser, D. E. (2015). 
    # Detection of atmospheric rivers: Evaluation and application of an
    # algorithm for global studies. Journal of Geophysical Research:
    # Atmospheres, 120(24), 12514-12535.
def detect_ARs(eIVT, nIVT, lat, lon, time,): 

    poleward_threshold = 50 # kg m^-1 s^-1, must be atleast this poleward
    dirDiff_threshold = np.pi / 4 # radians, not part of AR if has IVT difference from mean greater than this
    length_threshold = 2000 # km, must be atleast this long
    axThresLat, axThresLon = 4, 4 # threshold degrees latitude, longitude of difference between points on axis
    bottom_threshold = 100 # kh m^-1 s^-1 absolute minimum IVT for consideration of AR

    timePD = pd.to_datetime(time)
    # calculate the area of lat-lon grid vs. latitude, store in array
    area_vs_lat = np.zeros((lat.size,))
    R_earth = 6371 # radius of Earth in km
    delta_lon = float(np.abs(lon[2] - lon[1]))
    delta_lat = float(np.abs(lat[2] - lat[1]))
    for i in range(len(lat)):
        if abs(lat[i]) == 90:
            sgnPole = np.sign(lat[i])
            lat1 = np.deg2rad(sgnPole * 90)
            lat2 = np.deg2rad(sgnPole * (90 - delta_lat/2))
            area_vs_lat[i] = 2 * (np.sin(lat1) - np.sin(lat2))
        else:
            lat1 = np.deg2rad(lat[i] + delta_lat/2)
            lat2 = np.deg2rad(lat[i] - delta_lat/2)
            area_vs_lat[i] = np.sin(lat1) - np.sin(lat2)
    area_vs_lat = np.abs(area_vs_lat) * np.pi/180 * R_earth**2 * delta_lon

    # store information for calculating 5-month moving percentiles
    months = [pd.to_datetime(str(timePD[0].month) + '-' + str(timePD[0].year))]
    lens_months = []
    currentMonth = timePD[0].month
    # one entry for each month of data, assumes no months are skipped
    for i in range(1, len(timePD)):
        if timePD[i].month != currentMonth:
            currentMonth = timePD[i].month
            months.append(pd.to_datetime(str(timePD[i].month) + '-' + str(timePD[i].year)))
            lens_months.append(timePD[i-1].day)
    lens_months.append(timePD[-1].day)
    lens_months, months = np.array(lens_months), np.array(months)

    # calculate the IVT magnitude
    magIVT_allTime = np.zeros((len(timePD),eIVT[0].shape[0], eIVT[0].shape[1]))
    for i in range(len(timePD)):
        magIVT_allTime[i] = np.sqrt(np.power(eIVT[i], 2) + np.power(nIVT[i], 2))

    # calculate the percentiles
    percentiles_dict = {}
    des_percentiles = [85, 87.5, 90, 92.5, 95, 97.5]
    for des_perc in des_percentiles:
        percentiles = np.zeros((len(months),eIVT[0].shape[0], eIVT[0].shape[1]))
        if len(months) < 4:
            prctlFrame = np.percentile(magIVT_allTime[:], des_perc, axis=0)
            for i in range(len(months)): percentiles[i] = prctlFrame
        elif len(months) == 4:
            percentiles[0] = np.percentile(magIVT_allTime[:np.sum(lens_months[:3])], des_perc, axis=0)
            percentiles[-1] = np.percentile(magIVT_allTime[lens_months[1]:], des_perc, axis=0)
            prctlFrame = np.percentile(magIVT_allTime[:], des_perc, axis=0)
            for i in range(1,3): percentiles[i] = prctlFrame
        else:
            percentiles[0] = np.percentile(magIVT_allTime[:np.sum(lens_months[:3])], des_perc, axis=0)
            percentiles[1] = np.percentile(magIVT_allTime[:np.sum(lens_months[:4])], des_perc, axis=0)
            percentiles[-1] = np.percentile(
                magIVT_allTime[-1*np.sum(lens_months[-3:]):], des_perc, axis=0)
            percentiles[-2] = np.percentile(
                magIVT_allTime[-1*np.sum(lens_months[-4:]):], des_perc, axis=0)
            for i in range(2,len(months)-2):
                i1 = np.sum(lens_months[:i-2])
                i2 = i1 + np.sum(lens_months[i-2:i+3])
                percentiles[i] = np.percentile(magIVT_allTime[i1:i2], des_perc, axis=0)
        percentiles_dict[str(des_perc)] = percentiles

    # AR detection code
    structure = np.ones((3, 3), dtype = int)
    geodesic = pyproj.Geod(ellps='WGS84')

    # arrays to hold the data about the detected ARs
    ARdatetime, ARmeanMagIVT, ARmeanDirIVT = [], [], []
    ARlength, ARwidth, ARperimeterLon, ARperimeterLat = [], [], [], []
    ARbottomAxisLat, AReastAxisLon, ARtopAxisLat, ARwestAxisLon, ARlandfall = [], [], [], [], []

    reasonsForSkip = ['near equator', 'not poleward enough',
        'IVT direction different from elongation direction', 'too much variation in IVT direction',
        'not long enough', 'too wide']
    skipped = [0,0,0,0,0,0]

    for i in range(len(timePD)):
        # hold data for plotting
        validARcolsPost = np.zeros(eIVT[0].shape)
        axLatAll, axLonAll, sizeAx = [], [], []
        # magnitude of the IVT
        magIVT = np.copy(magIVT_allTime[i])
        # find which percentile frame to use
        prctlInd = np.searchsorted(
            months, pd.to_datetime(str(timePD[i].month) + '-' + str(timePD[i].year)))

        # iterate through percentiles
        for des_perc in des_percentiles:
            # true where threshold requirement is satisfied
            validARcols = (magIVT > percentiles_dict[str(des_perc)][prctlInd]) * (
                magIVT > bottom_threshold)
            # hold the coordinates of all groups of "true" values, labeled by number
            labeled, ncomponents = label(validARcols.astype(int), structure)

            # connect shapes the straddle longitude = 0deg
            if np.max(lon) - np.min(lon) >= 360 - delta_lon:
                for y in range(labeled.shape[0]):
                    if labeled[y, 0] > 0 and labeled[y, -1] > 0 and labeled[y,0] != labeled[y,-1]:
                        labeled[labeled == labeled[y, -1]] = labeled[y, 0]
                        ncomponents -= 1
                for j in range(1,ncomponents + 1):
                    if np.where(labeled==j)[0].size == 0:
                        labeled[labeled == labeled.max()] = j
            
            # iterate through each shape to consider further AR requirements
            j = 1
            while j <= ncomponents:
                ind_lat, ind_lon = np.where(labeled == j)
                # skip if if straddles the equator
                #if np.min(np.abs(lat[ind_lat])) < 2*delta_lat:
                #    skipped[0] += 1
                #    j += 1
                #    continue
                mean_lat = lat[ind_lat].mean()
                nIVT_t = np.array(nIVT[i])[ind_lat, ind_lon]
                # skip if no appreciable poleward IVT
                if nIVT_t.mean() * np.sign(mean_lat) < poleward_threshold:
                    skipped[1] += 1
                    j += 1
                    continue 
                eIVT_t = np.array(eIVT[i])[ind_lat, ind_lon]
                # direction of IVT in radians for each column
                dirIVT = np.arctan2(nIVT_t, eIVT_t)
                meanDir = dirIVT.mean()
                # find the two coordinates in the blob furthest apart
                inds_f = [0, -1] # index for ind_lon,lat of two furthest points
                ixLon, inLon = -1*np.argmax(np.flip(ind_lon))-1, np.argmin(ind_lon)
                zonal = True
                thruLon0 = (np.max(lon[ind_lon]) - np.min(lon[ind_lon])) >= 360 - 2*delta_lon
                if len(np.unique(ind_lon)) > len(np.unique(ind_lat)):
                    zonal = False
                    if not thruLon0: inds_f = [ixLon, inLon]
                    else:
                        ixLon = np.argmax(
                            np.array(lon[ind_lon]) * (np.array(lon[ind_lon]) < 180))
                        inLon = np.argmin(
                            np.array(lon[ind_lon]) + 360*(np.array(lon[ind_lon]) <= 180))
                        inds_f = [ixLon, inLon]
                # skip if mean direction differs from direction of elongation by > 45 degrees 
                fwd_azimuth, back_azimuth, distance = geodesic.inv(
                    lon[ind_lon[inds_f[0]]], lat[ind_lat[inds_f[0]]],
                    lon[ind_lon[inds_f[1]]], lat[ind_lat[inds_f[1]]])
                if np.abs(np.abs(meanDir) + np.pi/2 - np.abs(
                    np.deg2rad(fwd_azimuth))) > dirDiff_threshold:
                    skipped[2] += 1
                    j += 1
                    continue
                # skip if more than half differ by > direction threshold from the mean direction
                if (np.abs(dirIVT - meanDir) > dirDiff_threshold).sum() > len(ind_lat)/2:
                    skipped[3] += 1
                    j += 1
                    continue
                # divide into segments to calculate axis
                perimLat, perimLon = [], []
                perimLatFor, perimLatBack, perimLonFor, perimLonBack = [], [], [], []
                if zonal:
                    indNewSeg = (np.where((ind_lat[1:] - ind_lat[:-1]) > 0) + np.array([1]))[0]
                    if len(indNewSeg) < 2:
                        skipped[4] += 1
                        j += 1
                        continue
                    klat = ind_lat[0]
                    klon = ind_lon[np.argmax(magIVT[[klat],ind_lon[:indNewSeg[0]]])]
                    axLon, axLat = [klon], [klat]
                    for k in range(1, len(indNewSeg)):
                        klat = ind_lat[indNewSeg[k]]
                        klon = ind_lon[indNewSeg[k-1] + np.argmax(
                            magIVT[[klat],ind_lon[indNewSeg[k-1]:indNewSeg[k]]])]
                        perimLat.extend([lat[klat] - delta_lat/2, lat[klat] + delta_lat/2])
                        perimLonFor.extend(
                            [lon[ind_lon[indNewSeg[k-1]]] + delta_lon/2, lon[ind_lon[indNewSeg[k-1]]]\
                                + delta_lon/2])
                        perimLonBack.extend(
                            [lon[ind_lon[indNewSeg[k]-1]] - delta_lon/2, lon[ind_lon[indNewSeg[k]-1]]\
                                - delta_lon/2])
                        axLon.append(klon)
                        axLat.append(klat)
                    perimLat = perimLat + perimLat[::-1]
                    perimLon = perimLonFor + perimLonBack[::-1]
                    axisShaper = np.where(np.abs(
                        np.array(axLon[1:]) - np.array(axLon[:-1]))*delta_lon > axThresLon)[0] + 1
                    if len(axisShaper) > 0:
                        j += 1
                        continue
                else:
                    sortedByLon = np.asarray(sorted(zip(ind_lon, ind_lat)))
                    ind_lon, ind_lat = sortedByLon[:,0], sortedByLon[:,1]
                    if thruLon0:
                        indSplit = np.argmax(np.array(ind_lon[1:]) - np.array(ind_lon[:-1])) + 1
                        ind_lon[indSplit:] = ind_lon[indSplit:] - lon.size
                        sortedByLon = np.asarray(sorted(zip(ind_lon, ind_lat)))
                        ind_lon, ind_lat = sortedByLon[:,0], sortedByLon[:,1]
                    indNewSeg = (np.where((ind_lon[1:] - ind_lon[:-1]) > 0) + np.array([1]))[0]
                    if len(indNewSeg) < 2:
                        skipped[4] += 1
                        j += 1
                        continue
                    klon = ind_lon[0]
                    klat = ind_lat[np.argmax(magIVT[ind_lat[:indNewSeg[0]],[klon]])]
                    axLon, axLat = [klon], [klat]
                    for k in range(1, len(indNewSeg)):
                        klon = ind_lon[indNewSeg[k]]
                        klat = ind_lat[indNewSeg[k-1] + np.argmax(
                            magIVT[ind_lat[indNewSeg[k-1]:indNewSeg[k]],[klon]])]
                        perimLon.extend([lon[klon] - delta_lon/2, lon[klon] + delta_lon/2])
                        perimLatFor.extend(
                            [lat[ind_lat[indNewSeg[k-1]]] + delta_lat/2, lat[ind_lat[indNewSeg[k-1]]]\
                                + delta_lat/2])
                        perimLatBack.extend(
                            [lat[ind_lat[indNewSeg[k]-1]] - delta_lat/2, lat[ind_lat[indNewSeg[k]-1]]\
                                - delta_lat/2])
                        axLon.append(klon)
                        axLat.append(klat)
                    perimLat = perimLatFor + perimLatBack[::-1]
                    perimLon = perimLon + perimLon[::-1]
                    axisShaper = np.where(
                        np.abs(lat[axLat[1:]] - lat[axLat[:-1]]) > axThresLat)[0] + 1
                    if len(axisShaper) > 0:
                        j += 1
                        continue
                        
                lat_jump = np.array(np.abs(lat[axLat[1:]] - lat[axLat[:-1]]) < 3)
                lon_jump = np.array(np.abs(lon[axLon[1:]] - lon[axLon[:-1]]) < 3)

                length = np.abs(np.sum(lat_jump * lon_jump * estimate_dist_ll2(
                    np.array(lat[axLat[1:]]), np.array(lon[axLon[1:]]),
                    np.array(lat[axLat[:-1]]), np.array(lon[axLon[:-1]]))))
                # skip if length under threshold
                if length < length_threshold:
                    skipped[4] += 1
                    j += 1
                    continue

                area = np.sum(area_vs_lat[ind_lat])
                if length**2 / area < 2:
                    skipped[5] += 1
                    j += 1
                    continue

                sizeAx.append((len(axLatAll), len(axLatAll) + len(axLat)))
                axLatAll.extend(axLat)
                axLonAll.extend(axLon)
                validARcolsPost[ind_lat, ind_lon] = 1

                # add data for valid AR
                ARdatetime.append(timePD[i])
                ARmeanMagIVT.append(magIVT[ind_lat, ind_lon].mean())
                ARmeanDirIVT.append(meanDir)
                ARlength.append(length)
                ARwidth.append(area / length)

                if np.max(perimLon) >= 360-delta_lon and np.min(perimLon) <= delta_lon:
                    perimLon -= 360*(perimLon > 180)

                ARperimeterLon.append(np.array(perimLon))
                ARperimeterLat.append(np.array(perimLat))

                AReastAxisLon.append(lon[axLon[0]])
                ARbottomAxisLat.append(lat[axLat[0]])
                ARwestAxisLon.append(lon[axLon[-1]])
                ARtopAxisLat.append(lat[axLat[-1]])
                ARlandfall.append(np.any(
                    globe.is_land(lat[axLat], lon[axLon] - 360*(lon[axLon] > 180))))

                # zero out found ARs
                magIVT[ind_lat, ind_lon] = 0

                j += 1

    ARdatetime = np.array(ARdatetime)
    ARmeanMagIVT = np.array(ARmeanMagIVT)
    ARmeanDirIVT = np.array(ARmeanDirIVT)
    ARlength = np.array(ARlength)
    ARwidth = np.array(ARwidth)
    ARperimeterLon = np.array(ARperimeterLon, dtype=object)
    ARperimeterLat = np.array(ARperimeterLat, dtype=object)
    AReastAxisLon = np.array(AReastAxisLon)
    ARbottomAxisLat = np.array(ARbottomAxisLat)
    ARwestAxisLon = np.array(ARwestAxisLon)
    ARtopAxisLat = np.array(ARtopAxisLat)
    ARlandfall = np.array(ARlandfall)

    results = {
        'ARdatetime': ARdatetime,
        'ARmeanMagIVT': ARmeanMagIVT,
        'ARmeanDirIVT': ARmeanDirIVT,
        'ARlength': ARlength,
        'ARwidth': ARwidth,
        'ARperimeterLon': ARperimeterLon,
        'ARperimeterLat': ARperimeterLat,
        'AReastAxisLon': AReastAxisLon,
        'ARbottomAxisLat': ARbottomAxisLat,
        'ARwestAxisLon': ARwestAxisLon,
        'ARtopAxisLat': ARtopAxisLat,
        'ARlandfall': ARlandfall,
    }

    print('Recorded ' + str(len(ARdatetime)) + ' ARs!')
    print('Skipped enhanced IVT objects for the following reasons:')
    for reason, numSkip in zip(reasonsForSkip, skipped):
        print('-' + reason + ': ' + str(numSkip) + ' times')

    return results

In [3]:
# example downloading ERA5 IVT data
import cdsapi 
c = cdsapi.Client()
c.retrieve("reanalysis-era5-single-levels",
{
"variable": [
    "vertical_integral_of_eastward_water_vapour_flux",
    "vertical_integral_of_northward_water_vapour_flux"],
# see https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation for all possible
# variable names
"product_type": "reanalysis",
"year": ["1999"],
"month": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"],
"day": ["01","02","03","04","05","06","07","08","09","10","11",
        "12","13","14","15","16","17","18","19","20","21","22",
        "23","24","25","26","27","28","29","30","31"],
"time": ["00:00", "12:00",], # can adjust up to hourly
"area": ["0", "180", "-60", "-50"], # top lat, east lon (-180 -> 180), bottom lat, west lon
"grid": ["1.5", "1.5"], # can adjust as fine as 0.25 by 0.25 degrees
"format": "netcdf"
}, "IVT_1999.nc")

2022-12-02 16:27:38,568 INFO Welcome to the CDS
2022-12-02 16:27:38,568 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-12-02 16:27:38,906 INFO Request is queued
2022-12-02 16:27:40,150 INFO Request is running
2022-12-02 16:28:56,290 INFO Request is completed
2022-12-02 16:28:56,290 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data8/adaptor.mars.internal-1670009315.7337177-24610-6-12fef7bd-6345-469c-aa8e-cf8702781cf9.nc to IVT_1999.nc (9.9M)
2022-12-02 16:29:08,534 INFO Download rate 831.1K/s 


Result(content_length=10420472,content_type=application/x-netcdf,location=https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data8/adaptor.mars.internal-1670009315.7337177-24610-6-12fef7bd-6345-469c-aa8e-cf8702781cf9.nc)

In [4]:
# example usage of AR detector

# open dataset
data = xr.open_dataset('IVT_1999.nc')
data # display data info
time = data.variables['time']
# longitude array deg
lon = data.variables['longitude']
# latitude array deg
lat = data.variables['latitude'] 
# total column water vapor grid kg m^-2
#tcwv = data.variables['tcwv'] 
# eastward component of vertically integrated water vapor kg m^-1 s^-1
eIVT = data.variables['p71.162'] 
# northward component of vertically integrated water vapor kg m^-1 s^-1
nIVT = data.variables['p72.162']

# results is a dictionary with the following keys:
# 'ARdatetime', 'ARmeanMagIVT', 'ARmeanDirIVT', 'ARlength', 'ARwidth',
# 'ARperimeterLon', 'ARperimeterLat', 'AReastAxisLon', 'ARbottomAxisLat',
# 'ARwestAxisLon', 'ARtopAxisLat', 'ARlandfall'
results = detect_ARs(eIVT, nIVT, lat, lon, time,)

Recorded 560 ARs!
Skipped enhanced IVT objects for the following reasons:
-near equator: 0 times
-not poleward enough: 36509 times
-IVT direction different from elongation direction: 19057 times
-too much variation in IVT direction: 357 times
-not long enough: 9098 times
-too wide: 8 times
